In [1]:
import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image
import os
import json
import glob
from tqdm.notebook import tqdm

In [2]:
cwd = os.getcwd()

ROOT_DIR = os.path.join(cwd, "./data/case3-datasaur-photo/techosmotr/techosmotr/train")

labels = {}

for folder in os.listdir(ROOT_DIR):
    for file in os.listdir(os.path.join(ROOT_DIR, folder)):
        if file.endswith(".jpeg"):
            full_name = os.path.join(ROOT_DIR, folder, file)
            labels[full_name] = folder

files = labels.keys()

In [3]:
files

dict_keys(['/home/spectre/Projects/AI/datasour/AutoEye/./data/case3-datasaur-photo/techosmotr/techosmotr/train/0-correct/74651446.jpeg', '/home/spectre/Projects/AI/datasour/AutoEye/./data/case3-datasaur-photo/techosmotr/techosmotr/train/0-correct/77637890.jpeg', '/home/spectre/Projects/AI/datasour/AutoEye/./data/case3-datasaur-photo/techosmotr/techosmotr/train/0-correct/72126463.jpeg', '/home/spectre/Projects/AI/datasour/AutoEye/./data/case3-datasaur-photo/techosmotr/techosmotr/train/0-correct/75592104.jpeg', '/home/spectre/Projects/AI/datasour/AutoEye/./data/case3-datasaur-photo/techosmotr/techosmotr/train/0-correct/71057382.jpeg', '/home/spectre/Projects/AI/datasour/AutoEye/./data/case3-datasaur-photo/techosmotr/techosmotr/train/0-correct/73621855.jpeg', '/home/spectre/Projects/AI/datasour/AutoEye/./data/case3-datasaur-photo/techosmotr/techosmotr/train/0-correct/71506651.jpeg', '/home/spectre/Projects/AI/datasour/AutoEye/./data/case3-datasaur-photo/techosmotr/techosmotr/train/0-corre

In [4]:
import logging
from hydra import compose, initialize
from omegaconf import OmegaConf

from nn.autoeye.config import Config

if __name__ == "__main__":
    # global initialization
    initialize(version_base=None, config_path="nn/conf")
    cfg = compose(config_name="config")
    print(OmegaConf.to_yaml(cfg))
    log = logging.getLogger(__name__)
    Config.setup(cfg, log, train=True)

data:
  csv_files:
  - ./data/train.csv
  - ./data/test.csv
model:
  name: autoeye
  backbone: dino_vision
  version: 1.0
optim:
  beta1: 0.9
  beta2: 0.95
  weight_decay: 0.1
hyper:
  pretrained: true
  batch_size: 16
  dropout: 0.2
  lr: 0.003
  epochs: 10
  eval_iters: 1
  use_amp: false



In [5]:
from nn.autoeye.model import AutoEye


transform_image = T.Compose([T.ToTensor(), T.Resize(520), T.CenterCrop(518), T.Normalize([0.5], [0.5])])
model = AutoEye().to(Config.device)

Using cache found in /home/spectre/.cache/torch/hub/facebookresearch_dinov2_main
/home/spectre/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/spectre/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/spectre/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitb14/dinov2_vitb14_linear4_head.pth" to /home/spectre/.cache/torch/hub/checkpoints/dinov2_vitb14_linear4_head.pth
100%|██████████| 14.7M/14.7M [00:02<00:00, 6.52MB/s]


In [6]:
def load_image(img: str) -> torch.Tensor:
    """
    Load an image and return a tensor that can be used as an input to DINOv2.
    """
    img = Image.open(img)

    transformed_img = transform_image(img)[:3].unsqueeze(0)

    return transformed_img


def compute_embeddings(files: list) -> dict:
    """
    Create an index that contains all of the images in the specified list of files.
    """
    all_embeddings = {}

    with torch.no_grad():
        for i, file in enumerate(tqdm(files)):
            embeddings = model.backbone(load_image(file).to(Config.device))

            all_embeddings[file] = (
                np.array(embeddings[0].cpu().numpy()).reshape(1, -1).tolist()
            )

    with open("all_embeddings.json", "w") as f:
        f.write(json.dumps(all_embeddings))

    return all_embeddings

In [8]:
embeddings = compute_embeddings(files)

  0%|          | 0/6585 [00:00<?, ?it/s]

In [12]:
y = [labels[file] for file in files]
embedding_list = list(embeddings.values())
X = np.array(embedding_list).reshape(-1, 1000)

In [15]:
from PIL import Image
import pandas as pd
from sklearn import svm
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score

clf = svm.SVC(gamma=0.0001, kernel="poly", C=100)
clf.fit(X, y)
scores = cross_val_score(clf, X, y, cv=5)
scores

array([0.91571754, 0.92027335, 0.90888383, 0.92634776, 0.92027335])

In [16]:
import pickle

filename = "./models/classifier.pickle"

# save model
pickle.dump(clf, open(filename, "wb"))

In [20]:
source = "./data/case3-datasaur-photo/techosmotr/techosmotr/test"
allfiles = os.listdir(source)
test_data = {"file_index": [], "class": []}
# iterate on all files to move them to destination folder
for f in allfiles:
    # f = os.listdir(source)[random.randint(0, 300)]
    src_path = os.path.join(source, f)

    input = load_image(os.path.join(source, f))
    embedding = model.backbone(input.to(Config.device))

    prediction = clf.predict(embedding[0].cpu().detach().numpy().reshape(1, -1))
    if prediction == "0-correct":
        prediction = 0
    else:
        prediction = 1

    test_data["file_index"].append(f.split(".")[0])
    test_data["class"].append(prediction)

/home/spectre/Projects/AI/datasour/AutoEye/venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/spectre/Projects/AI/datasour/AutoEye/venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) w

In [21]:
sub = pd.DataFrame(test_data)
sub.to_csv("./output.csv", index=False)